In [114]:
import pandas as pd
import os

In [272]:
path = 'C:/Users/lifei/Documents/waterproj/dam_manage_info' #path 변수에 경로 지정

In [291]:
f_list = os.listdir(path) #디렉토리 안에 뭐 있는지 보여줌
print(f_list)

['201901.csv', '201902.csv', '201903.csv', '201904.csv', '201905.csv', '201906.csv', '201907.csv', '201908.csv', '201909.csv', '201910.csv', '201911.csv', '201912.csv', '202001.csv', '202002.csv', '202003.csv', '202004.csv', '202005.csv', '202006.csv', '202007.csv', '202008.csv', '202009.csv', '202010.csv', '202011.csv', '202012.csv', '202101.csv', '202102.csv', '202103.csv', '202104.csv', '202105.csv', '202106.csv', '202107.csv', '202108.csv', '202109.csv', '202110.csv', '202111.csv', '202112.csv', '202201.csv', '202202.csv', '202203.csv', '202204.csv', '202205.csv', '202206.csv', '202207.csv', '202208.csv', '202209.csv', '202210.csv', '202211.csv', '202212.csv']


In [294]:
# df = pd.read_csv(path + "/" + '202207.csv', sep = ',')

In [295]:
df = pd.DataFrame()
for i in range(len(f_list)):
    temp_df = pd.read_csv(path + "/" + f_list[i], sep = ',')
    df = pd.concat([df, temp_df])
    # print(f_list[i])

In [296]:
df

,dam_nm,obsr_de,rsvr_wal_hg,rain_qy,inflow_qy,tot_dcwtr_qy,rsvwt_qy,dam_rsvwt_rt
0,충주,20190101,139.8000,0.0,35.381,104.637,2147.7650,78.10
1,횡성,20190101,177.3600,0.0,0.373,1.614,71.9450,82.80
2,소양강,20190101,182.5600,0.0,9.063,14.959,1864.7390,64.30
3,안동,20190101,154.2300,0.0,1.122,16.978,929.9470,74.50
4,임하,20190101,154.8700,0.0,0.000,8.555,374.2330,62.90
...,...,...,...,...,...,...,...,...
646,부안,20221231,38.8510,0.0,0.085,0.739,35.6002,70.78
647,섬진강,20221231,175.7370,0.0,3.099,4.715,81.4090,17.47
648,주암(본),20221231,93.7882,0.0,0.572,5.195,131.0083,28.67
649,주암(조),20221231,81.0920,0.0,1.113,8.249,69.4086,27.76


In [297]:
# df[df['dam_nm'] == '충주']  

In [298]:
# df.to_csv("dam_total.csv")

In [299]:
# 불필요 컬럼 삭제
df.drop(labels = 'rsvr_wal_hg',axis = 1, inplace=True )
# df.drop(labels = 'H',axis = 1, inplace=True )

In [300]:
# 유효저수용량 데이터 불러들이기
df_valid_capa = pd.read_csv('C:/Users/lifei/Documents/waterproj/valid_capa.csv', sep = ',')

# 필요한 컬럼들만 취하기
df_valid_capa = df_valid_capa[['dam_nm','valid_rsvwt_cpcty_qy']]

In [301]:
# 기존 df 와 df_valid_capa merge(join) 하기
df = pd.merge(left = df, right=df_valid_capa, how = "inner", on = "dam_nm" ) #dam_nm 기준으로

In [302]:
# 지역 데이터 불러들이기
df_region = pd.read_csv('C:/Users/lifei/Documents/waterproj/region.csv', sep = ',')

In [303]:
# 기존 df 와 df_region merge(join) 하기
df = pd.merge(left = df, right=df_region, how = "inner", on = "dam_nm" ) #dam_nm 기준으로

In [304]:
# 기후 데이터 읽어들이기
df_climate = pd.read_csv('C:/Users/lifei/Documents/waterproj/climate.csv', sep = ',')

In [305]:
# 날짜 컬럼의 데이터타입을 Datetime 타입으로 변경
df['obsr_de'] = pd.to_datetime(df['obsr_de'], format='%Y%m%d')
df_climate['obsr_de'] = pd.to_datetime(df_climate['obsr_de'])

In [306]:
df = pd.merge(left = df, right=df_climate, how = "inner", on = ['region_nm','obsr_de'] ) #dam_nm 기준으로 조인

In [307]:
df.head(5)

,dam_nm,obsr_de,rain_qy,inflow_qy,tot_dcwtr_qy,rsvwt_qy,dam_rsvwt_rt,valid_rsvwt_cpcty_qy,region_nm,avg_temp,avg_wind(m/s),avg_hmdty(%),tot_insolation(MJ/m2)
0,충주,2019-01-01,0.0,35.381,104.637,2147.765,78.1,1789,충주,-4.2,1.7,60.3,NaN
1,충주,2019-01-02,0.0,25.577,104.592,2140.938,77.9,1789,충주,-6.6,1.1,57.9,NaN
2,충주,2019-01-03,0.0,15.902,104.623,2133.272,77.6,1789,충주,-6.5,0.8,54.9,NaN
3,충주,2019-01-04,0.0,25.901,104.612,2126.472,77.3,1789,충주,-4.1,0.6,58.3,NaN
4,충주,2019-01-05,0.0,26.022,104.590,2119.683,77.1,1789,충주,-2.8,1.1,52.1,NaN


In [308]:
# 초단위 데이터를 일단위로 변경함
df['inflow_qy'] = df['inflow_qy']*86400
df['tot_dcwtr_qy'] = df['tot_dcwtr_qy']*86400


In [309]:
# 합계 일사량 데이터가 없는 지역 확인
df_climate[df_climate['tot_insolation(MJ/m2)'].isnull()]['region_nm'].unique()

array(['충주', '추풍령', '안동', '대구', '홍천', '보령', '부안', '임실', '장수', '장흥', '영주',
       '영천', '합천', '밀양'], dtype=object)

In [310]:
# 합계 일사량 데이터는 결측치가 많아서 컬럼을 삭제함
df.drop(labels = 'tot_insolation(MJ/m2)',axis = 1, inplace=True )

In [311]:
# 최종 합쳐진 데이터 저장
df.to_csv("raw_merge_final.csv")

## Scaling 실행(MinMax, Standard)

In [312]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [313]:
# rain_qy 의 값이 " "  것을 0 으로 변경 
df.loc[df["rain_qy"] == " ", "rain_qy"] = 0

In [314]:
# rain_qy, valid_rsvwt_cpcty_qy 의 데이터 타입을 object 에서 float 으로 변경(스케일링을 위한 사전 전처리 작업임)

df['rain_qy']=df['rain_qy'].astype(float)
df['valid_rsvwt_cpcty_qy']=df['valid_rsvwt_cpcty_qy'].astype(float)


In [315]:

colnames = ['rain_qy', 'inflow_qy', 'tot_dcwtr_qy', 'rsvwt_qy','dam_rsvwt_rt', 'valid_rsvwt_cpcty_qy', 'avg_temp','avg_wind(m/s)', 'avg_hmdty(%)']
mm_scalar = MinMaxScaler()
mm_scalar.fit(df[colnames])
mm_scaled_data = mm_scalar.transform(df[colnames])


new_colnames = [i+'_mms' for i in colnames] 
# dataframe으로 minmax scaling 된 것과 변환전 데이터 같이 나타내기
df_mms = pd.concat([df, pd.DataFrame(mm_scalar.fit_transform(df[colnames]), columns=new_colnames)], axis=1)

In [316]:
std_scalar = StandardScaler()
std_scalar.fit(df[colnames])
std_scaled_data = mm_scalar.transform(df[colnames])


new_colnames = [i+'_stds' for i in colnames] 
# dataframe으로 minmax scaling 된 것과 변환전 데이터 같이 나타내기
df_stds = pd.concat([df, pd.DataFrame(std_scalar.fit_transform(df[colnames]), columns=new_colnames)], axis=1)

In [317]:
# 불필요 컬럼 삭제
df_mms.drop(['rain_qy', 'inflow_qy', 'tot_dcwtr_qy', 'rsvwt_qy','dam_rsvwt_rt', 'valid_rsvwt_cpcty_qy', 'avg_temp','avg_wind(m/s)', 'avg_hmdty(%)'], axis =1,inplace=True)
df_stds.drop(['rain_qy', 'inflow_qy', 'tot_dcwtr_qy', 'rsvwt_qy','dam_rsvwt_rt', 'valid_rsvwt_cpcty_qy', 'avg_temp','avg_wind(m/s)', 'avg_hmdty(%)'], axis =1,inplace=True)

In [318]:
df_mms.head(5)

,dam_nm,obsr_de,region_nm,rain_qy_mms,inflow_qy_mms,tot_dcwtr_qy_mms,rsvwt_qy_mms,dam_rsvwt_rt_mms,valid_rsvwt_cpcty_qy_mms,avg_temp_mms,avg_wind(m/s)_mms,avg_hmdty(%)_mms
0,충주,2019-01-01,충주,0.0,0.006487,0.020812,0.888710,0.789687,0.941024,0.266393,0.188889,0.512883
1,충주,2019-01-02,충주,0.0,0.004691,0.020803,0.885885,0.787664,0.941024,0.217213,0.122222,0.483436
2,충주,2019-01-03,충주,0.0,0.002918,0.020810,0.882713,0.784631,0.941024,0.219262,0.088889,0.446626
3,충주,2019-01-04,충주,0.0,0.004750,0.020807,0.879900,0.781598,0.941024,0.268443,0.066667,0.488344
4,충주,2019-01-05,충주,0.0,0.004773,0.020803,0.877090,0.779575,0.941024,0.295082,0.122222,0.412270


In [319]:
df_mms.head(5)

,dam_nm,obsr_de,region_nm,rain_qy_mms,inflow_qy_mms,tot_dcwtr_qy_mms,rsvwt_qy_mms,dam_rsvwt_rt_mms,valid_rsvwt_cpcty_qy_mms,avg_temp_mms,avg_wind(m/s)_mms,avg_hmdty(%)_mms
0,충주,2019-01-01,충주,0.0,0.006487,0.020812,0.888710,0.789687,0.941024,0.266393,0.188889,0.512883
1,충주,2019-01-02,충주,0.0,0.004691,0.020803,0.885885,0.787664,0.941024,0.217213,0.122222,0.483436
2,충주,2019-01-03,충주,0.0,0.002918,0.020810,0.882713,0.784631,0.941024,0.219262,0.088889,0.446626
3,충주,2019-01-04,충주,0.0,0.004750,0.020807,0.879900,0.781598,0.941024,0.268443,0.066667,0.488344
4,충주,2019-01-05,충주,0.0,0.004773,0.020803,0.877090,0.779575,0.941024,0.295082,0.122222,0.412270


In [320]:
df_mms.to_csv("data_mms.csv")
df_stds.to_csv("data_stds.csv")